In [ ]:
import numpy as np
import torchtext

from os import path
from utils import read_json_file

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Simple class for validating L2 of the ML-EAT
class attribute_word:
    def __init__(self, word, vector):
        self.word = word
        self.vector = vector
    
    def __str__(self):
        return self.word
    
    def __repr__(self):
        return self.word
    
    def set_T_associations(self, T_vec_list):
        self.T_associations = [cosine_similarity(self.vector, t) for t in T_vec_list]
    
    def set_T_mean(self):
        self.T_mean = np.mean(self.T_associations)
    
    def get_T_mean(self):
        return self.T_mean

In [ ]:
# Simple class-based validation of L2 measurements using Glove embeddings
# -----------------------
# Path constants
JSON_PATH = './eat_stimuli/text_stimuli'
DF_PATH = './'

# ML-EAT constants
PERMUTATIONS = 10_000

# Load embedding
embedding = torchtext.vocab.GloVe(name="840B", dim=300)

# Store results in a list of dataframes
result_dfs = []

# Run ML-EAT on each set of WEAT stimuli
for i in range(1, 11):
    stimuli = read_json_file(path.join(JSON_PATH, f'weat_{i}.json'))

    A_vec_list = [embedding.get_vecs_by_tokens([word]).numpy().squeeze() for word in stimuli['A']]
    B_vec_list = [embedding.get_vecs_by_tokens([word]).numpy().squeeze() for word in stimuli['B']]
    X_vec_list = [embedding.get_vecs_by_tokens([word]).numpy().squeeze() for word in stimuli['X']]
    Y_vec_list = [embedding.get_vecs_by_tokens([word]).numpy().squeeze() for word in stimuli['Y']]

    A_dict_X = {word: attribute_word(word, vector) for word, vector in zip(stimuli['A'], A_vec_list)}
    B_dict_X = {word: attribute_word(word, vector) for word, vector in zip(stimuli['B'], B_vec_list)}
    A_dict_Y = {word: attribute_word(word, vector) for word, vector in zip(stimuli['A'], A_vec_list)}
    B_dict_Y = {word: attribute_word(word, vector) for word, vector in zip(stimuli['B'], B_vec_list)}

    for word in stimuli['A']:
        A_dict_X[word].set_T_associations(X_vec_list)
        A_dict_X[word].set_T_mean()
        A_dict_Y[word].set_T_associations(Y_vec_list)
        A_dict_Y[word].set_T_mean()
    
    for word in stimuli['B']:
        B_dict_X[word].set_T_associations(X_vec_list)
        B_dict_X[word].set_T_mean()
        B_dict_Y[word].set_T_associations(Y_vec_list)
        B_dict_Y[word].set_T_mean()

    X_effect = (np.mean([A_dict_X[word].get_T_mean() for word in stimuli['A']]) - np.mean([B_dict_X[word].get_T_mean() for word in stimuli['B']])) / np.std(np.concatenate([np.array([A_dict_X[word].get_T_mean() for word in stimuli['A']]), np.array([B_dict_X[word].get_T_mean() for word in stimuli['B']])]), ddof=1)
    Y_effect = (np.mean([A_dict_Y[word].get_T_mean() for word in stimuli['A']]) - np.mean([B_dict_Y[word].get_T_mean() for word in stimuli['B']])) / np.std(np.concatenate([np.array([A_dict_Y[word].get_T_mean() for word in stimuli['A']]), np.array([B_dict_Y[word].get_T_mean() for word in stimuli['B']])]), ddof=1)

    print(f'WEAT {i} X effect: {X_effect}')
    print(f'WEAT {i} Y effect: {Y_effect}')

In [ ]:
# Programmatic Proof that L2 generalizes the SC-WEAT effect size
# -----------------------
# Path constants
JSON_PATH = './eat_stimuli/text_stimuli'
DF_PATH = './'

# ML-EAT constants
PERMUTATIONS = 10_000

# Load embedding
embedding = torchtext.vocab.GloVe(name="840B", dim=300)

# Store results in a list of dataframes
result_dfs = []

# Run ML-EAT on each set of WEAT stimuli
for i in range(1, 11):
    stimuli = read_json_file(path.join(JSON_PATH, f'weat_{i}.json'))

    A_vec_list = [embedding.get_vecs_by_tokens([word]).numpy().squeeze() for word in stimuli['A']]
    B_vec_list = [embedding.get_vecs_by_tokens([word]).numpy().squeeze() for word in stimuli['B']]
    X_vec_list = [embedding.get_vecs_by_tokens([word]).numpy().squeeze() for word in stimuli['X']][:1] # Only one word in X
    Y_vec_list = [embedding.get_vecs_by_tokens([word]).numpy().squeeze() for word in stimuli['Y']][:1] # Only one word in Y

    # SC-WEAT effect size
    x = X_vec_list[0]
    X_A = [cosine_similarity(x, a) for a in A_vec_list]
    X_B = [cosine_similarity(x, b) for b in B_vec_list]
    eff = (np.mean(X_A) - np.mean(X_B)) / np.std(np.concatenate([X_A, X_B]), ddof=1)
    print(f'SC-WEAT {i} X effect size: {eff}')

    y = Y_vec_list[0]
    Y_A = [cosine_similarity(y, a) for a in A_vec_list]
    Y_B = [cosine_similarity(y, b) for b in B_vec_list]
    eff = (np.mean(Y_A) - np.mean(Y_B)) / np.std(np.concatenate([Y_A, Y_B]), ddof=1)
    print(f'SC-WEAT {i} Y effect size: {eff}')

    # ML-EAT L2 effect sizes - same code as above
    A_dict_X = {word: attribute_word(word, vector) for word, vector in zip(stimuli['A'], A_vec_list)}
    B_dict_X = {word: attribute_word(word, vector) for word, vector in zip(stimuli['B'], B_vec_list)}
    A_dict_Y = {word: attribute_word(word, vector) for word, vector in zip(stimuli['A'], A_vec_list)}
    B_dict_Y = {word: attribute_word(word, vector) for word, vector in zip(stimuli['B'], B_vec_list)}

    for word in stimuli['A']:
        A_dict_X[word].set_T_associations(X_vec_list)
        A_dict_X[word].set_T_mean()
        A_dict_Y[word].set_T_associations(Y_vec_list)
        A_dict_Y[word].set_T_mean()
    
    for word in stimuli['B']:
        B_dict_X[word].set_T_associations(X_vec_list)
        B_dict_X[word].set_T_mean()
        B_dict_Y[word].set_T_associations(Y_vec_list)
        B_dict_Y[word].set_T_mean()

    X_effect = (np.mean([A_dict_X[word].get_T_mean() for word in stimuli['A']]) - np.mean([B_dict_X[word].get_T_mean() for word in stimuli['B']])) / np.std(np.concatenate([np.array([A_dict_X[word].get_T_mean() for word in stimuli['A']]), np.array([B_dict_X[word].get_T_mean() for word in stimuli['B']])]), ddof=1)
    Y_effect = (np.mean([A_dict_Y[word].get_T_mean() for word in stimuli['A']]) - np.mean([B_dict_Y[word].get_T_mean() for word in stimuli['B']])) / np.std(np.concatenate([np.array([A_dict_Y[word].get_T_mean() for word in stimuli['A']]), np.array([B_dict_Y[word].get_T_mean() for word in stimuli['B']])]), ddof=1)

    print(f'ML-EAT L2 {i} X effect: {X_effect}')
    print(f'ML-EAT L2 {i} Y effect: {Y_effect}')